In [4]:
import numpy as np
from scipy import linalg
from scipy.sparse import diags, csc_matrix, kron
from scipy.sparse import linalg as spla
import matplotlib.pyplot as plt
from pathlib import Path

## Helper functions

In [5]:
def printmatrix(mat, **kwargs):
    fp = kwargs.get("fp", 3)    # number of floating points
    bs = kwargs.get("bs", 8)   # number of blank-spaces
    format_dict = {
        'float_kind': lambda x: f"{x:>{bs}.{fp}f}",
        'bool': lambda x: False if x == 0 else True,
        }
    formatter = kwargs.get("formatter", format_dict)
    linewidth = kwargs.get("linewidth", None)
    if linewidth is None:
        linewidth = kwargs.get("lw", 100)
    with np.printoptions(formatter=formatter, linewidth=linewidth):
        print(mat)

## Create folder for figures

In [6]:
# make directory if not already present
assignment_dir = Path.cwd() # directory for this working directory (if using notebook, it will be the direcotry of notebook)
figures_dir = assignment_dir / "figures" # figures directory
figures_dir.mkdir(parents=True, exist_ok=True) # create folder and parent directories, if not already existing.

# Exercise 1 Design ODE solver

## Subtask 1.1) Checking if unique solution exists

We can use Picard-Lindelöf theorem to determine if a unique solution exists in an interval around the initial value $(t, y) = (0, \delta)$.
we have the initial value problem:
$$\begin{align}
    y' = y^2 - y^3, \quad y(0) = \delta, \quad 0\leq t \leq \frac{2}{\delta}
\end{align}$$

Before we go further, the RHS $f(t,y) = y^2 - y^3$ has to be Lipschitz continuous in $u$ and $t$:
$$\begin{align}
    \mathcal{D} = \left\{(t,y) :  |y-\delta| \leq a, 0\leq t \leq \frac{2}{\delta}\right\}
\end{align}$$

$f$ is Lipschitz continous in the domain $\mathcal{D}$ if:
* there exists $L\geq 0 $ that for all $(t,y),(t, y^*) \in \mathcal{D}$
$$\begin{align}
    |f(t,y) - f(t, y^*)| \leq L|y - y^*|
\end{align}$$

We can determine $L$ if $f$ is differentiable w.r.t. $y$ in the domain $\mathcal{D}$, $f_y = \frac{\partial f}{\partial y}$, and $f_y$ is bounded:
$$\begin{align}
    L = \max_{(t,y) \in \mathcal{D}}{|f_y(t,y)|}
\end{align}$$


### Determining the Lipschitz continuity

$f(t,y) = y^2 - y^3$ is a polynium, and is therefore continous and differentiable:
$$\begin{align}
    f_y &= 2y - 3y^2 \\
\end{align}$$
we see that $f_y$ is bounded in the interval $0\leq t\leq 2/\delta$ and $\delta \leq y \leq y(2/\delta)$

$$\begin{align}
    |f(t,y) - f(t, \delta)| &\leq L|y - \delta| \\
    |(y^2 - y^3) - (\delta^2 - \delta^3)| &\leq L|y - \delta| \\
    L &= \max_{y\in\mathcal{D}}{|2y - 3y^2|}
\end{align}$$

### Picard-Lindelöf
Now that we ahve confirmed Lipschitz continuity on the domain $\mathcal{D}$, the Picard-Lindelöf theorem says that there exists a unique solution in the range $t\in[0,T^*]$
$$\begin{align}
    T^* = \min{(\delta, 0 + a/S)} \\
    a = \max_{(t,y) \mathcal{D}}|y - \delta|,\:\: S = \max_{(t,y)\in{\mathcal{D}}}|f(t,y)|
\end{align}$$

## Subtask 1.2) solving RK23

The exercise references seciton 5.5 in Ensiger-Karraup & Thomsen (2018), which is a walkthrough of RK-23.
The method for constructing a Runge-Kutta, is

$$\begin{align*}
    \xi_1 &= y_n \\
    \xi_2 &= y_n + a_{2,1} f(t_n, \xi_1)\\
    \xi_3 &= y_n + a_{3,1}f(t_n, \xi_1) + a_{3,2}f(t_n + c_2h, \xi_2) \\
    y_{n+1} &= y_n + h(b_1 f(t_n, \xi_1) + b_2f(t_n + c_2h,\xi_2) + b_3 f(t_n + c_3h, \xi) ) \\
        &= y_n + h\sum_{i=1}^3 b_i f(t_i, \xi_i), \quad t_i = t_n + c_ih \\
    e_{n+1} &= h\sum_{i=1}^3 d_i f(t_i, \xi_i)
\end{align*}$$

Then we construct a Buthcers scheme
$$\begin{align*}
    \begin{array}{c|ccc}
    c_1     & 0         &           &   \\
    c_2     & a_{2,1}   &       0   &   \\
    c_3     & a_{3,1}   & a_{3,2}   & 0 \\\hline
    \vec{b} & b_1       & b_2       & b_3 \\\hline
    \vec{d} & d_1       & d_2       & d_3
    \end{array}
\end{align*}$$

We need to have 
$$ c_1 = 0, \quad c_2 = a_{2,1}, \quad c_3 = a_{3,1} + a_{3,2} $$

In [ ]:
def _step_rk23(f, tn, yn, h, **kwargs):
    reps = kwargs.get("reps", 1e-4)
    aeps = kwargs.get("aeps", 1e-6)
    delta = kwargs.get("delta", 2e-2)
    
    # from p. 51, table 5.3 in Engsig-Kaarup & Thomsen (2018)
    C = np.array([0, 1/2, 1])
    A = np.array([[0,   0, 0],
                  [1/2, 0, 0],
                  [-1,  2, 0]])
    b = np.array([1/6, 1/3, 1/6]) # 3rd order
    d = np.array([1/12, -1/6, 1/12]) # 2nd order
    
    xi = np.empty(shape=(3,))
    ti = tn + C*h
    # F = f(ti, xi)
    xi[0] = yn
    xi[1] = yn + A[1,0]*h*f(tn, xi[0])
    xi[2] = yn + h*(A[2,0]*f(tn, xi[0]) + A[2,1]*f(tn+C[1]*h, xi[1]))
    y_new = yn + h*b@f(ti, xi)
    
    error = h*d@f(ti, xi)
    
    success = (error < reps*np.abs(y_new) + aeps) # True/false 
    
    return y_new, success, error
    
    
    

SyntaxError: invalid syntax (1548412735.py, line 17)